In [34]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import copy
from tqdm import tqdm


# Neural stuff

In [35]:
sigm = lambda x: 1/(1+np.exp(-x))

In [36]:
class Layer:
    
    
    def __init__(self,NC,NN,ActFun,rate=0.05): # Jugar con la tasa de mutacion
        
        self.NC = NC
        self.NN = NN
        self.ActFunc = ActFun
        self.rate = rate
        
        self.W = np.random.uniform( -10.,10.,(self.NC,self.NN) )
        self.b = np.random.uniform( -10.,10.,(1,self.NN) )
        
    def Activation(self,x):
        z = np.dot(x,self.W) + self.b
        return self.ActFunc( z )[0]
    
    def Mutate(self):
    
        #self.W += np.random.normal( loc=0., scale=self.rate, size=(self.NC,self.NN))
        #self.b += np.random.normal( loc=0., scale=self.rate, size=(1,self.NN))
        
        self.W += np.random.uniform( -self.rate, self.rate, size=(self.NC,self.NN))
        self.b += np.random.uniform( -self.rate, self.rate, size=(1,self.NN))

In [37]:
def GetBrain():
    l0 = Layer(1,5,sigm)
    l1 = Layer(5,1,sigm)
    #l2 = Layer(2,1,sigm)
    Brain = [l0,l1]
    return Brain    

# Estructura del individuo

In [38]:
class Robot:
    
    def __init__(self, dt, Layers, Id=0):
        
        self.Id = Id
        self.dt = dt
        
        
        self.r = np.random.uniform([0.,0.])
        theta = 0.
        self.v = np.array([1.*np.cos(theta),1.*np.sin(theta)])

        
        # Capacidad o aptitud del individuo
        self.Fitness = np.inf
        self.Steps = 0

        # Brain
        self.Layers = Layers
        
    def GetR(self):
        return self.r
    
    def Evolution(self):
        self.r += self.v*self.dt # Euler integration (Metodos 2)

        # Cada generación regreamos el robot al origin
        # Y volvemos a estimar su fitness
    def Reset(self):
        self.Steps = 0.
        self.r = np.array([0.,0.])
        self.Fitness = np.inf   
    
    # Aca debes definir que es mejorar en tu proceso evolutivo
    def SetFitness(self,threshold=0.85):
        penal=0
        if abs(self.r[0]) > 1:
            penal += 11
            
        elif self.Activation[0]<threshold and abs(self.r[0])<0.9:
            penal += 10   # Incrementamos la penalización por alta activación cerca de cero
        self.Fitness = penal
        
    def BrainActivation(self, x, threshold=0.85):
         # Forward pass - la información fluye por el modelo hacia adelante
        for i in range(len(self.Layers)):         
            if i == 0:
                output = self.Layers[i].Activation(x)
            else:
                output = self.Layers[i].Activation(output)

        self.Activation = np.round(output, 4)

        # Cambiamos el vector velocidad y aplicamos la penalización
        
        if self.Activation[0] > threshold and 0.9 <= abs(self.r[0]) < 1:
            self.v = -self.v
            self.Activation*=0.9

        # Actualizamos la penalización y el fitness
        

        return self.Activation

    # Aca mutamos (cambiar de parametros) para poder "aprender"
    def Mutate(self):
        for i in range(len(self.Layers)):
            self.Layers[i].Mutate()
    
    # Devolvemos la red neuronal ya entrenada
    def GetBrain(self):
        return self.Layers

In [39]:
def GetRobots(N):
    
    Robots = []
    
    for i in range(N):
        
        Brain = GetBrain()
        r = Robot(dt,Brain,Id=i)
        Robots.append(r)
        
    return Robots

In [40]:
dt = 0.1
t = np.arange(0.,12.,dt)
Robots = GetRobots(10)

In [41]:
def GetPlot():
    
    fig = plt.figure(figsize=(8,4))
    ax = fig.add_subplot(1,2,1)
    ax1 = fig.add_subplot(1,2,2)
    
    ax.set_xlim(-1.,1.)
    ax.set_ylim(-1.,1.)
 
    return ax,ax1

# Time evolution

In [42]:
def TimeEvolution(Robots,e,Plot=True):
    
  
    for it in range(t.shape[0]):
        
        if Plot:
        
            clear_output(wait=True)
        
            ax,ax1 = GetPlot()
            ax1.set_ylim(0.,1.)
        
            ax.set_title('t = {:.3f}'.format(t[it]))
        
        Activation = np.zeros(len(Robots))
        
        for i,p in enumerate(Robots):
            p.Evolution()
         
            # Activacion cerebral
            Act = p.BrainActivation(p.GetR()[0])[0]
            Activation[i] = Act
            # Region donde aumentamos los pasos para el fitness
            
                
            if Plot and i < 5: # Solo pintamos los primeros 5, por tiempo de computo
                ax.scatter(p.r[0],p.r[1],label='Id: {}, Steps: {:.0f}'.format(p.Id,p.Steps))
                ax.quiver(p.r[0],p.r[1],p.v[0],p.v[1])
                
        # Pintamos la activaciones de los primeros 5
        
        if Plot:
            ax1.plot(np.arange(0,len(Robots[:5]),1),Activation[:5],marker='o',color='b',label='Activation')
            ax1.axhline(y=0.7,color='r')
        
        if Plot:
        
            ax.legend(loc=0)  
            ax1.legend(loc=0)
            plt.show()
            time.sleep(0.001)

# Algoritmo evolutivo

In [43]:
def Genetic(Robots, epochs=200, Plot=True, Plottime=False):
    N = int(0.7 * len(Robots))

    FitVector = np.array([])
    x = np.linspace(-1, 1, 20)
    Act = np.zeros_like(x)

    weights_evolution = []
    biases_evolution = []

    for e in range(int(epochs)):
        for p in Robots:
            p.Reset()
            p.Mutate()

        TimeEvolution(Robots, e, Plottime)

        for i, p in enumerate(Robots):
            p.SetFitness()

        scores = [(p.Fitness, p) for p in Robots]
        scores.sort(key=lambda x: x[0], reverse=False)

        Temp = [r[1] for i, r in enumerate(scores) if i < N]

        for i, r in enumerate(Robots):
            j = i % N
            Robots[i] = copy.deepcopy(Temp[j])

        best_fitness = scores[0][0]
        best_bot = scores[0][1]

        best_weights = best_bot.Layers[0].W
        best_biases = best_bot.Layers[0].b

        weights_evolution.append(best_weights)
        biases_evolution.append(best_biases)

        FitVector = np.append(FitVector, best_fitness)

        for i in range(len(x)):
            Act[i] = best_bot.BrainActivation(x[i])

        clear_output(wait=True)

        print('Época:', e)
        print('Último Fitness:', FitVector[-1])

        

    return best_bot, FitVector



In [44]:
Robots = GetRobots(10)
Best, FitVector = Genetic(Robots,Plot=True,Plottime=True) # Apagar Plottime para el entrenamiento
#A veces, por alguna razón, si lo corres,la primera vez no sirve, pero si lo corres de nuevo aprende rápido.
#masomenos después de la tercer época.

KeyboardInterrupt: 